## Jack Robbins
## CMP-262
## Professor Binowski
## Project 4 - Sentiment Analysis of Tweets

Question: What is the overall sentiment of Elon Musk's most recent 500 tweets?                                                                                                                                                                              
Solution: We will discover this by pulling Elon Musk's most recent 500 tweets with the twitter API, cleaning them, and performing sentiment analysis on them.

In [59]:
# Import all needed packages
import tweepy
import geopy
import preprocessor as pre
from wordcloud import WordCloud
from IPython.display import Image
import requests
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [38]:
# Get Elon Musk's tweets using the twitter V2 API
bearer_token = "AAAAAAAAAAAAAAAAAAAAANQXkgEAAAAAbvaTHt2KFGHL08%2F7KU81QHvGvaw%3DlLc5URsjp9Ni2BSBnKjqeX4W6v8OMFP3G29dsCgD4f6tJqWy2A"
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
elon = client.get_user(username='elonmusk',user_fields=['description', 'public_metrics'])
elon_tweets = client.get_users_tweets(id=elon.data.id, max_results=5)
# But this has a max of 99 tweets, we need a paginator object to get over 100 tweets

In [53]:
# Create a paginator object, then iterate through it to get 500 tweets
paginator = tweepy.Paginator(client.get_users_tweets, elon.data.id, max_results = 100)
elon_tweets = []
for tweet in paginator.flatten(limit=500):
    elon_tweets.append(tweet)

In [56]:
# Now extract the text out of the tweets
elon_tweet_text = []
for tweet in elon_tweets:
    elon_tweet_text.append(tweet.text)

elon_tweet_text

['@KimDotcom 🤣',
 'Great goal by Argentina! 🇦🇷🇦🇷🇦🇷 https://t.co/WIs9ocfPcz',
 'At World Cup right now https://t.co/CG7zMMxSjE',
 'Follow The World Cup on Twitter!\nhttps://t.co/SaTzwDzBuR',
 '@EricRWeinstein Temp suspension due to prior doxxing action by this account. Will be lifted shortly.',
 '@neontaster Agreed, a high priority software update for Twitter is showing the reason for account suspension or deboosting. \n\nRules should be equally &amp; consistently applied.',
 '@KimDotcom Maybe it’s a coincidence 🤷\u200d♂️',
 '@BasedBeffJezos 🔥🔥🔥',
 '@MartinKulldorff @DrJBhattacharya The tide is turning fast for the Faucists',
 '@DavidSacks @mtaibbi Because they’re in on it',
 '@cb_doge @RepAdamSchiff 🤣',
 '@MilMileBattery @Steam @TheWitcherMS @Tesla 🔥🔥',
 'Hallelujah!! https://t.co/i2FyvXPIHO',
 '@teslaownersSV @Tesla A real concern!',
 '@mayemusk @larryelder It’s fine',
 '@ShefVaidya @Twitter @verified Will investigate',
 '@DavidSacks They abandon even the pretense of noble principles 

In [58]:
# Now let's use the preprocessor to clean the tweets
pre.set_options(pre.OPT.URL, pre.OPT.RESERVED)
clean_tweets = []
for tweet in elon_tweet_text:
    clean_tweets.append(pre.clean(tweet))
clean_tweets

['@KimDotcom 🤣',
 'Great goal by Argentina! 🇦🇷🇦🇷🇦🇷',
 'At World Cup right now',
 'Follow The World Cup on Twitter!',
 '@EricRWeinstein Temp suspension due to prior doxxing action by this account. Will be lifted shortly.',
 '@neontaster Agreed, a high priority software update for Twitter is showing the reason for account suspension or deboosting. Rules should be equally &amp; consistently applied.',
 '@KimDotcom Maybe it’s a coincidence 🤷\u200d♂️',
 '@BasedBeffJezos 🔥🔥🔥',
 '@MartinKulldorff @DrJBhattacharya The tide is turning fast for the Faucists',
 '@DavidSacks @mtaibbi Because they’re in on it',
 '@cb_doge @RepAdamSchiff 🤣',
 '@MilMileBattery @Steam @TheWitcherMS @Tesla 🔥🔥',
 'Hallelujah!!',
 '@teslaownersSV @Tesla A real concern!',
 '@mayemusk @larryelder It’s fine',
 '@ShefVaidya @Twitter @verified Will investigate',
 '@DavidSacks They abandon even the pretense of noble principles in their zeal to keep their monopoly on information seen by the public',
 '@KohlmanSweeney There will

Twitter Keys
API key: WJtZRifH8tI8wVBDhXTLQmifz
API Key Secret: UnQbOCCw4QDbQH713osugCO2A2sCi0IKvYQjMv6OM7bFXbv16a
Bearer Token: AAAAAAAAAAAAAAAAAAAAANQXkgEAAAAAbvaTHt2KFGHL08%2F7KU81QHvGvaw%3DlLc5URsjp9Ni2BSBnKjqeX4W6v8OMFP3G29dsCgD4f6tJqWy2A
Access Token: 1604139786251517960-B1oAQ1YQVNQjGmMtMEeJOI5wXL19e4
Access Token Secret: uvSIZmwDrq8Zub0prZgijtAr4JtLHUKxFAQ6wO8Ayng16